In [1]:
!apt-get install -y ffmpeg

!pip install praw
!pip install streamlit
!pip install pyngrok
!pip install yt-dlp
!pip install streamlink
!pip install moviepy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.2/531.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 20.5 M

# check the urls

In [2]:
%%writefile app.py

import praw
import streamlit as st

# Reddit API credentials
reddit = praw.Reddit(client_id='DAOso5_7CHzXzdtd-070fg',
                     client_secret='JtdGFRDM10avSQFYthzYUQNfLeI8rQ',
                     user_agent='Mental Health')

# Streamlit Application
st.title("Extract DASHplaylist.mpd from Reddit Posts")
username = st.text_input("Enter Reddit Username:")

if username:
    try:
        user = reddit.redditor(username)
        submissions = user.submissions.new(limit=10)  # Fetch recent submissions

        dash_urls = []
        for submission in submissions:
            # Debug: Print media content
            st.write(f"Post Title: {submission.title}")
            st.write(submission.media)  # Show media dictionary for debugging

            if submission.is_video and submission.media:
                # Explicitly extract dash_url
                reddit_video = submission.media.get("reddit_video", {})
                dash_url = reddit_video.get("dash_url")
                if dash_url:
                    dash_urls.append({
                        "title": submission.title,
                        "dash_url": dash_url,
                    })

        # Display results
        if dash_urls:
            for dash in dash_urls:
                st.write(f"**Post Title:** {dash['title']}")
                st.video(dash['dash_url'])  # Display the video in Streamlit
        else:
            st.warning("No DASH URLs found for this user.")

    except Exception as e:
        st.error(f"Error: {e}")


Writing app.py


# combined audio video

In [2]:
%%writefile app.py

# method 1
import praw
import streamlit as st
import requests
import os
from moviepy.editor import VideoFileClip, AudioFileClip

# Initialize Reddit API
reddit = praw.Reddit(client_id='DAOso5_7CHzXzdtd-070fg',
                     client_secret='JtdGFRDM10avSQFYthzYUQNfLeI8rQ',
                     user_agent='Mental Health')


def download_video(video_url, save_path):
    """Download a video from the given URL and save it locally."""
    try:
        video_data = requests.get(video_url)
        with open(save_path, 'wb') as f:
            f.write(video_data.content)
        return save_path
    except Exception as e:
        st.write(f"Error downloading video: {e}")
        return None

def download_audio(audio_url, save_path):
    """Download audio from the given URL and save it locally."""
    try:
        audio_data = requests.get(audio_url)
        with open(save_path, 'wb') as f:
            f.write(audio_data.content)
        return save_path
    except Exception as e:
        st.write(f"Error downloading audio: {e}")
        return None

def combine_video_audio(video_path, audio_path, output_path):
    """Combine video and audio into one file."""
    try:
        video_clip = VideoFileClip(video_path)
        audio_clip = AudioFileClip(audio_path)

        # Set the audio of the video clip
        video_clip = video_clip.set_audio(audio_clip)

        # Write the final output video
        video_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

        return output_path
    except Exception as e:
        st.write(f"Error combining video and audio: {e}")
        return None

def get_user_posts_with_videos(username, max_items=10):
    """Fetch latest posts with videos from a Reddit user's profile."""
    try:
        # Attempt to fetch the user's posts
        user = reddit.redditor(username)

        post_data = []
        for submission in user.submissions.new(limit=max_items):
            videos = []

            # Check if the post is a direct video
            if submission.is_video:
                # Get the URL of the hosted video (Reddit video URL)
                video_url = submission.media['reddit_video']['fallback_url']

                # Dynamically generate the audio URL by replacing the resolution part with _AUDIO_128.mp4
                audio_url = video_url.split("DASH_")[0] + "DASH_AUDIO_128.mp4"
                videos.append({'video_url': video_url, 'audio_url': audio_url})

            # Only add posts with videos
            if videos:
                post_data.append({"text": submission.title, "videos": videos})

        return post_data

    except praw.exceptions.RedditAPIException as e:
        st.error(f"Error fetching data from user '{username}': {e}")
        return []
    except Exception as e:
        st.error(f"Error fetching user data: {e}")
        return []


# Streamlit App
st.title("Reddit User Video Extractor with Audio")

username = st.text_input("Enter Reddit Username (e.g., username):")
if username:
    st.write(f"Fetching video data for Reddit user {username}...")
    posts_with_videos = get_user_posts_with_videos(username, max_items=10)

    if posts_with_videos:
        for i, post in enumerate(posts_with_videos, start=1):
            st.subheader(f"Post {i}:")
            st.write(post["text"])

            # Download video and audio, then combine
            for vid_data in post["videos"]:
                st.write(f"Downloading and combining video and audio for Post {i}...")

                # Download Video
                video_path = f"video_{i}.mp4"
                downloaded_video_path = download_video(vid_data['video_url'], video_path)

                # Download Audio
                audio_path = f"audio_{i}.mp4"
                downloaded_audio_path = download_audio(vid_data['audio_url'], audio_path)

                # Combine video and audio if both are downloaded
                if downloaded_video_path and downloaded_audio_path:
                    combined_video_path = f"combined_video_{i}.mp4"
                    final_video = combine_video_audio(downloaded_video_path, downloaded_audio_path, combined_video_path)

                    if final_video:
                        st.video(final_video)
                        os.remove(final_video)  # Clean up after displaying
                        os.remove(downloaded_video_path)  # Clean up after displaying
                        os.remove(downloaded_audio_path)  # Clean up after displaying
                    else:
                        st.warning("Could not combine video and audio.")
                else:
                    st.warning("Could not download video or audio.")
    else:
        st.warning("No videos found in this user's posts!")


Writing app.py


# Trying ffmpeg

In [2]:
%%writefile app.py

import praw
import streamlit as st
import requests
import os
import subprocess

# Set the FFmpeg binary path explicitly for Colab
os.environ["FFMPEG_BINARY"] = "/usr/bin/ffmpeg"

# Initialize Reddit API
reddit = praw.Reddit(client_id='DAOso5_7CHzXzdtd-070fg',
                     client_secret='JtdGFRDM10avSQFYthzYUQNfLeI8rQ',
                     user_agent='Mental Health')


def download_video(video_url, save_path):
    """Download a video from the given URL and save it locally."""
    try:
        video_data = requests.get(video_url)
        with open(save_path, 'wb') as f:
            f.write(video_data.content)
        return save_path
    except Exception as e:
        st.write(f"Error downloading video: {e}")
        return None


def download_audio(audio_url, save_path):
    """Download audio from the given URL and save it locally."""
    try:
        audio_data = requests.get(audio_url)
        with open(save_path, 'wb') as f:
            f.write(audio_data.content)
        return save_path
    except Exception as e:
        st.write(f"Error downloading audio: {e}")
        return None


def combine_video_audio(video_path, audio_path, output_path):
    """Combine video and audio into one file using FFmpeg."""
    try:
        # FFmpeg command to combine video and audio
        ffmpeg_command = [
            "/usr/bin/ffmpeg",
            "-i", video_path,  # Input video file
            "-i", audio_path,  # Input audio file
            "-c:v", "libx264",  # Use libx264 codec for video
            "-c:a", "aac",  # Use AAC codec for audio
            "-strict", "experimental",  # Allow experimental AAC encoding
            "-shortest",  # Use the shortest length (video or audio) to determine the output length
            output_path  # Output file path
        ]

        # Run FFmpeg command
        subprocess.run(ffmpeg_command, check=True)
        return output_path
    except Exception as e:
        st.write(f"Error combining video and audio: {e}")
        return None


def get_user_posts_with_videos(username, max_items=10):
    """Fetch latest posts with videos from a Reddit user's profile."""
    try:
        # Attempt to fetch the user's posts
        user = reddit.redditor(username)

        post_data = []
        for submission in user.submissions.new(limit=max_items):
            videos = []

            # Check if the post is a direct video
            if submission.is_video:
                # Get the URL of the hosted video (Reddit video URL)
                video_url = submission.media['reddit_video']['fallback_url']

                # Dynamically generate the audio URL by replacing the resolution part with _AUDIO_128.mp4
                audio_url = video_url.split("DASH_")[0] + "DASH_AUDIO_128.mp4"
                videos.append({'video_url': video_url, 'audio_url': audio_url})

            # Only add posts with videos
            if videos:
                post_data.append({"text": submission.title, "videos": videos})

        return post_data

    except praw.exceptions.RedditAPIException as e:
        st.error(f"Error fetching data from user '{username}': {e}")
        return []
    except Exception as e:
        st.error(f"Error fetching user data: {e}")
        return []


# Streamlit App
st.title("Reddit User Video Extractor with Audio")

username = st.text_input("Enter Reddit Username (e.g., username):")
if username:
    st.write(f"Fetching video data for Reddit user {username}...")
    posts_with_videos = get_user_posts_with_videos(username, max_items=10)

    if posts_with_videos:
        for i, post in enumerate(posts_with_videos, start=1):
            st.subheader(f"Post {i}:")
            st.write(post["text"])

            # Check if video and/or audio are available and process accordingly
            for vid_data in post["videos"]:
                st.write(f"Processing video and audio for Post {i}...")

                # Download Video
                video_path = f"video_{i}.mp4"
                downloaded_video_path = download_video(vid_data['video_url'], video_path)

                # Download Audio
                audio_path = f"audio_{i}.mp4"
                downloaded_audio_path = download_audio(vid_data['audio_url'], audio_path)

                # If both video and audio are available, combine them
                if downloaded_video_path and downloaded_audio_path:
                    combined_video_path = f"combined_video_{i}.mp4"
                    final_video = combine_video_audio(downloaded_video_path, downloaded_audio_path, combined_video_path)

                    if final_video:
                        st.video(final_video)
                        os.remove(final_video)  # Clean up after displaying
                        os.remove(downloaded_video_path)  # Clean up after displaying
                        os.remove(downloaded_audio_path)  # Clean up after displaying
                    else:
                        # st.warning("Could not combine video and audio.")
                        if downloaded_video_path:
                            # If only the video is available, display the video directly
                            st.video(downloaded_video_path)
                            os.remove(downloaded_video_path)  # Clean up after displaying
                        elif downloaded_audio_path:
                            # If only the audio is available, display the audio directly
                            st.audio(downloaded_audio_path)
                            os.remove(downloaded_audio_path)  # Clean up after displaying
                else:
                    st.warning("No video or audio found.")

                for file in os.listdir():
                    if file.endswith(".mp4"):
                        os.remove(file)
    else:
        st.warning("No videos found in this user's posts!")


Writing app.py


In [3]:
# Import ngrok
from pyngrok import ngrok

# Set your authtoken
ngrok.set_auth_token("2ohUKqk37HcGbvwN0s8Y1E2WNxE_39z1gVF3bYq9vFSEm7Wzq") # Replace YOUR_AUTHTOKEN with your actual authtoken

# Kill any existing ngrok processes
ngrok.kill()

# Start Streamlit with nohup
!nohup streamlit run app.py &

# Create a public URL with ngrok to access the app
public_url = ngrok.connect(addr='8501')
print(f"Public URL: {public_url}")

nohup: appending output to 'nohup.out'
Public URL: NgrokTunnel: "https://d86a-34-125-50-200.ngrok-free.app" -> "http://localhost:8501"


In [6]:
ngrok.kill()